In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd

from shapely.geometry import Point

In [2]:
df_subway_ridership_iter = pd.read_csv('../../data/raw/subway_turnstile/MTA_Subway_Hourly_Ridership__Beginning_February_2022_20240324.csv', chunksize=10000)

In [4]:
list_df = []
for chunk in df_subway_ridership_iter:
    df_tmp = chunk.loc[chunk.loc[:,'borough'] == 'Manhattan']
    df_tmp.loc[:,'transit_timestamp'] = pd.to_datetime(df_tmp.loc[:,'transit_timestamp'])
    df_tmp = df_tmp.loc[df_tmp.loc[:,'transit_mode'] == 'subway']
    df_tmp = df_tmp.drop(columns=['borough','Georeference','station_complex'])
    list_df.append(df_tmp)

df_ridership = pd.concat(list_df, ignore_index=True)

C:\Users\seung\AppData\Local\Temp\ipykernel_14156\3886346492.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tmp.loc[:,'transit_timestamp'] = pd.to_datetime(df_tmp.loc[:,'transit_timestamp'])
C:\Users\seung\AppData\Local\Temp\ipykernel_14156\3886346492.py:4: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df_tmp.loc[:,'transit_timestamp'] = pd.to_datetime(df_tmp.loc[:,'transit_timestamp'])
C:\Users\seung\AppData\Local\Temp\ipykernel_14156\3886346492.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy o

In [7]:
df_ridership.loc[:,'transit_hour'] = df_ridership.loc[:,'transit_timestamp'].dt.hour
df_ridership.loc[:,'station_complex_id'] = df_ridership.loc[:,'station_complex_id'].astype(int)

C:\Users\seung\AppData\Local\Temp\ipykernel_14156\3654702989.py:2: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df_ridership.loc[:,'station_complex_id'] = df_ridership.loc[:,'station_complex_id'].astype(int)


In [8]:
df_stations = df_ridership.loc[:,['station_complex_id','latitude','longitude']].groupby('station_complex_id', as_index=False).mean()

In [9]:
df_stations

,station_complex_id,latitude,longitude
0,8,40.764812,-73.973350
1,9,40.764664,-73.980660
2,10,40.759899,-73.984139
3,13,40.745495,-73.988692
4,14,40.741302,-73.989341
...,...,...,...
116,623,40.719502,-74.000076
117,624,40.713980,-74.008676
118,625,40.718606,-73.987485
119,628,40.710347,-74.006712


In [21]:
def categorize_time(hour):
    if hour in range(0,6):
        return 'ridership_late_night'
    elif hour in range(6,10):
        return 'ridership_morning'
    elif hour in range(10,16):
        return 'ridership_midday'
    elif hour in range(16,20):
        return 'ridership_evening'
    else:
        return 'ridership_night'

def create_annual_data(df, year):
    df_annual = df.loc[df.loc[:,'transit_timestamp'].dt.year == year]
    df_agg = df_annual.drop(['fare_class_category',
                             'transit_mode',
                             'payment_method',
                             'transit_timestamp',
                             'latitude',
                             'longitude',
                             'transfers'], axis=1).groupby(['station_complex_id','transit_hour'], as_index=False).mean()
    
    df_agg.loc[:,'time_category'] = df_agg.loc[:,'transit_hour'].apply(categorize_time)

    df_agg = df_agg.groupby(['station_complex_id','time_category'], as_index=False).sum().drop(['transit_hour'], axis=1)

    df_pivot = df_agg.pivot(index='station_complex_id', columns='time_category', values='ridership')
    df_pivot = df_pivot.reset_index()
    df_pivot.columns.name = ''

    df_pivot = df_pivot.merge(df_stations, on='station_complex_id')
    points = [Point(x,y) for x, y in zip(df_pivot.loc[:,'longitude'], df_pivot.loc[:,'latitude'])]
    gdf = gpd.GeoDataFrame(df_pivot, geometry=points)
    
    return gdf.drop(['latitude','longitude'], axis=1)

In [22]:
gdf_2022 = create_annual_data(df_ridership, 2022)
gdf_2023 = create_annual_data(df_ridership, 2023)

In [24]:
gdf_2022.to_file('../../data/processed/traffic/2022_ridership.geojson', driver='GeoJSON')
gdf_2023.to_file('../../data/processed/traffic/2023_ridership.geojson', driver='GeoJSON')